# A STFT (short-time Fourier transform)
Aldo André Díaz-Salazar, PhD, EE

## Objetivo
Neste notebook, vamos introduzir o estudo e processamento de sinais 2D, como por exemplo imagens, ou também a STFT (uma extensão da Transformada de Fourier).

Para isso, usuaremos como exemplo a telefonia DTMF, para tentar entender como os números discados são decodificados.

## A sinalização DTMF - Dual-tone multi-frequency

A sinalização DTMF é a maneira como os telefones analógicos enviam o número discado por um usuário para a central telefônica.
Isso foi antes de todas as redes digitais e celulares serem a norma.
Mas, o método ainda é usado para seleção de opções durante a chamada ("pressione 4 para falar com o atendimento ao cliente...").

O mecanismo é bastante inteligente:
O teclado do telefone é organizado em uma grade $4 \times 3$ e cada botão é associado a *duas* frequências de acordo com esta tabela:

|            | **1209 Hz** | **1336 Hz** | **1477 Hz** |
|------------|:-----------:|:-----------:|:-----------:|
| **697 Hz** |      1      |      2      |      3      |
| **770 Hz** |      4      |      5      |      6      |
| **852 Hz** |      7      |      8      |      9      |
| **941 Hz** |      *      |      0      |      #      |

As frequências na tabela foram escolhidas para serem "coprimas".
Em outras palavras, nenhuma frequência é múltipla de qualquer outra, o que reduz a probabilidade de detectar erroneamente os sinais recebidos devido à interferência.
Quando um botão é pressionado, as duas frequências correspondentes são geradas simultaneamente e enviadas pela linha.
Por exemplo, se o dígito '1' for pressionado, o sinal gerado será:

$$
    x(t) = \sin(2\pi\cdot 1209\cdot t) + \sin(2\pi\cdot697\cdot t)
$$

As especificações oficiais para o padrão DTMF estipulam ainda que:

 * Cada tom deve ter pelo menos 65 ms de duração.
 * Tons correspondentes a dígitos sucessivos devem ser separados por um intervalo silencioso de pelo menos 65 ms.

Neste notebook, construiremos um decodificador DTMF baseado na Transformada Discreta de Fourier.
É claro que aqui usaremos exclusivamente sinais de tempo discreto, então, se o relógio do sistema for $F_s$, cada tom DTMF terá a forma:

$$
    x[n] = \sin\left(2\pi\,\left(\frac{f_l}{F_s}\right)\, n\right) + \sin\left(2\pi\,\left(\frac{f_h}{F_s}\right)\,n\right)
$$

Pergunta: Quais números foram discados?

In [1]:
# first our usual bookkeeping
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../..')
from lib.myPlotLib import *
from lib.myAudioProcessingLib import *
from lib.myDFTLib import *

%matplotlib inline

In [2]:
data = np.load('dtmf.npz')
x  = data['x']
fs = int(data['fs'])
# dialtone = data['dialtone']

In [3]:
play(x, fs)

Vamos começar pensando no decodificador.
Podemos usar a seguinte estratégia:

 * Dividir o sinal em tons de dígitos individuais observando a posição dos silêncios.
 * Passar uma DFT nos tons de dígitos
 * Observar os picos da transformada de Fourier e recuperar o número discado

Aqui, estamos assumindo que temos o sinal inteiro na memória, ou seja, realizaremos o processamento do sinal todo de uma vez só em *batch* (lote).
Porém, um sistema mais prático decodificaria o sinal de entrada conforme ele chega, amostra por amostra (processamento em tempo real) e, de fato, isso ser assunto de futuras pesquisas suas.

Para dividir o sinal, a ideia é observar a energia local em pequenas janelas: quando o sinal estiver em silêncio, nós o separaremos.

Antes de fazer isso; vamos plotar e observar os dados brutos primeiro (conheça seu dataset!)

In [4]:
# Sua resposta aqui

<img src='./figures/fig15.png' width='800'>

<img src='./figures/fig16.png' width='800'>

<img src='./figures/fig17.png' width='800'>

In [5]:
# Sua resposta aqui

<img src='./figures/fig01.png' width='800'>

<img src='./figures/fig02.png' width='400'>

In [6]:
# Sua resposta aqui

<img src='./figures/fig03.png' width='800'>

<img src='./figures/fig04.png' width='400'>

In [7]:
# Sua resposta aqui

<img src='./figures/fig05.png' width='800'>

<img src='./figures/fig06.png' width='400'>

In [8]:
# Sua resposta aqui

<img src='./figures/fig07.png' width='800'>

<img src='./figures/fig08.png' width='400'>

Pergunta: Quais foram os dígitos discados?

## Espectrograma

Agora iremos analisar os recursos da STFT, para isso vamos precisar da seguinte função:

In [9]:
def stft_basic(x, w, H=8, only_positive_frequencies=False):
    '''
    Compute a basic version of the discrete short-time Fourier transform (STFT)
    Source: https://www.audiolabs-erlangen.de/resources/MIR/FMP/C2/C2_STFT-Basic.html

    Args:
        x (np.ndarray): Signal to be transformed
        w (np.ndarray): Window function
        H (int): Hopsize (Default value = 8)
        only_positive_frequencies (bool): Return only positive frequency part of spectrum (non-invertible)
            (Default value = False)

    Returns:
        X (np.ndarray): The discrete short-time Fourier transform
    '''
    N = len(w)
    L = len(x)
    M = np.floor((L - N) / H).astype(int) + 1
    X = np.zeros((N, M), dtype='complex')
    for m in range(M):
        x_win = x[m*H : m*H + N] * w
        X_win = myDFT(x_win)
        X[:, m] = X_win

    if only_positive_frequencies:
        K = N // 2 if N % 2 == 0 else N // 2 + 1 # If N is odd return N/2 + 1 points
        X = X[0:K, :]
        
    return X

Começamos com uma STFT como os seguintes parâmetros iniciais:

In [ ]:
# Short-Time Fourier Transform (STFT)
L = 256 # window length
w = np.ones(L) # Funcao de janela, por exemplo Hanning: signal.hann(L)
H = 1 # hope size

X = stft_basic(x, w, H, only_positive_frequencies=True)

Na STFT, duas informações básicas são:

In [ ]:
# Informacoes basicas da STFT
N_FFT_POINTS = X.shape[0] # igual a L/2
N_STFT_SEGMENTS = X.shape[1] # igual a 2N/L
print('N_FFT_POINTS = ' + str(N_FFT_POINTS))
print('N_STFT_SEGMENTS = ' + str(N_STFT_SEGMENTS))

Agora plote o espectro de magnitude da STFT e verifique o resultado

In [ ]:
Xmag = ... # STFT magnitude

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 4))
plt.imshow(Xmag, origin='lower', aspect='auto', cmap='gist_heat')
plt.xlabel(r'$m$') # Index (frames)
plt.ylabel(r'$k$') # Index (frequency)
plt.xlim([0, N])
ax.set_xticks([0, N // H], ['0', r'$N$'])
ax.set_yticks([0, L // 2], ['0', r'$L/2$'])

figureFormat(ax, fig)

<img src='./figures/fig18.png' width='800'>

Para observar melhor a potência do sinal (em dBs), usemos uma escala logaritmica

In [ ]:
Xmaglog = ... # STFT magnitude

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 4))
plt.imshow(Xmaglog, origin='lower', aspect='auto', cmap='gist_heat')
plt.xlabel(r'$m$') # Index (frames)
plt.ylabel(r'$k$') # Index (frequency)
plt.xlim([0, N])
ax.set_xticks([0, N // H], ['0', r'$N$'])
ax.set_yticks([0, L // 2], ['0', r'$L/2$'])

figureFormat(ax, fig)

<img src='./figures/fig19.png' width='800'>

Podemos rotular melhor os eixos colocando oo tempo (na horizontal) e a frequência (na vertical)

In [ ]:
# Sua resposta aqui

<img src='./figures/fig12.png' width='800'>

Agora, plote a STFT fazendo ajuste de parâmetros para os seguintes espectros e comente as diferenças:
 - Wideband
 - Middleband
 - Narrowband

In [ ]:
# Sua resposta aqui

<img src='./figures/fig09.png' width='800'>

In [ ]:
# Sua resposta aqui

<img src='./figures/fig10.png' width='800'>

In [ ]:
# Sua resposta aqui

<img src='./figures/fig11.png' width='800'>

## Time-frequency tiling

Vamos agora com um sinal mais complexo e divertido de analisar, a voz humana!

In [ ]:
x, fs = audioread('../../sounds/freddy.wav')
if x.ndim > 1:
    x = x.sum(axis=1) / 2.0 # mono audio conversion

play(x, fs)

Rodamos a STFT com um tamanho de janela inicial de $L = 128$ amostras.

Plote o sinal e seu espectrograma

In [ ]:
# Sua resposta aqui

Projete uma nova STFT para melhorar a resolucao do espectrograma, tendo como parâmetros:

- 36.6ms analysis window (27.3Hz frequency bins)
- 4.6ms shifts

In [ ]:
# Sua resposta aqui

<img src='./figures/fig13.png' width='800'>